# Limpieza de los datos

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from collections import Counter
import re
import nltk
nltk.download(['punkt','stopwords','wordnet','words'])
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#import unidecode

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cmejia\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
from nltk import *
from nltk.corpus import *

### input_file_path es la ruta donde se encuentran los archivos de texto plano

In [3]:
base_path =  Path("~").expanduser().resolve()
#base_path = Path.cwd().expanduser().resolve()
#input_file_path  = base_path / 'datasets/papers-txt/'
input_file_path = Path('E:/OneDrive - CELSIA S.A E.S.P/Maestría/Almacenamiento/Tika/Tika/limpiados')
#datasetOut =base_path / "datasets/salidas_procesamiento/"
#datasetOut_freq = base_path / "datasets/salidas_freq/"

In [4]:
input_file_path

WindowsPath('E:/OneDrive - CELSIA S.A E.S.P/Maestría/Almacenamiento/Tika/Tika/limpiados')

# Preprocesamiento

In [5]:
def fileSize(fileIn):
    size=os.stat(fileIn).st_size
    return size

def wordsCount(contenido):
    totWwords=contenido.split()
    return len(totWwords)

def cleanWordCount(contenido):
    contenido =re.sub('((f|ht)tp(s?)://)?(.*)[.][a-z]+(((\/.*(\/?)){1,}?)(.*([.].*)?))?',' ',contenido) # Eliminar las URL
    #contenido =re.sub('REFERENCES (\S|\w)+',' ',contenido) # Eliminar la bibliografia
    contenido =re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*',' ',contenido) # Eliminar los correos
    contenido =re.sub('\[[a-zA-Z0-9\,\. ]+\]',' ',contenido) # Eliminar cualquier contenido entre corchetes
    contenido =re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',contenido) # Eliminar cualquier contenido entre paréntesis
    contenido =re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\'|\’|\`)',' ',contenido) # Eliminar abreviaciones, apostrofes y guion
    contenido =re.sub('(á|à|ä)','a',contenido) # Reemplazar a acentuada
    contenido =re.sub('(é|è|ë)','e',contenido) # Reemplazar e acentuada
    contenido =re.sub('(í|ì|ï)','i',contenido) # Reemplazar i acentuada
    contenido =re.sub('(ó|ò|ö)','o',contenido) # Reemplazar o acentuada
    contenido =re.sub('(ú|ù|ü)','u',contenido) # Reemplazar u acentuada
    contenido =re.sub('-|\u2212|\u2012|\u2013|\u2014|\u2015',' ',contenido) # Reemplazar el guión
    contenido =re.sub('[^a-zA-Z]',' ',contenido) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    contenido =re.sub('(a-z|A-Z){1,2}',' ',contenido) # Eliminar palabras o números de un caracter de longitud 
    contenido =re.sub('((\w*)?xyz(\w*)?)',' ',contenido) # Eliminar palabras que tiene la cadena xyz 
    contenido =re.sub('((\w*)?abc(\w*)?)',' ',contenido) # Eliminar palabras que tiene la cadena abc
    contenido =re.sub(' +',' ',contenido) # Eliminar espacios en blanco
    contenidoDuplicado =re.findall(r'(\w*?(\w)\2{2,}.*?)',contenido) #Elminar carecteres repetidos
    i=0
    longDuplicados=len(contenidoDuplicado)
    while i <longDuplicados:
        pattern= "\w*" + contenidoDuplicado [i][0] +"\w*"
        contenido=re.sub(pattern,' ',contenido) # Eliminar las URL
        i=i+1
    #contenido =re.sub('[^A-Za-z0-9.,_%+-\(\)\[\]\´\'\`]',' ',contenido)
    #contenido =re.sub('\[(0-9)+\]',' ',contenido)    
    #totWordDepurado = Counter(map(str, contenido.split()))
    #outputFile= open(datasetOut, 'w', encoding='UTF-8')
    #outputFile.write(contenido)
    #outputFile.close()
    totWwords=contenido.split()
    setWords = set(totWwords)
    #print("Total de palabras {}".format(len(totWwords)))
    #print("Total de palabras después del pre-procesamiento: {}".format(totWordDepurado))
    return len(totWwords),contenido,setWords

# Procesamiento

In [6]:
def lang_ratio(input):
    lang_ratio = {}
    tokens = wordpunct_tokenize(input)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        word_set = set(words)
        common_elements = word_set.intersection(stopwords_set)
        lang_ratio[language] = len(common_elements)
    return lang_ratio

def detect_language(input):
    ratios = lang_ratio(input)
    language = max(ratios, key = ratios.get)
    return language

Conjunto de las stop words que serán eliminadas ya que no aportan valor:

In [7]:
stopWords = stopwords.words('english')

Palabras que considermos StopWords que no estan incluidas en el conjunto descargado:

In [8]:
listoStopWords = ['www','https','html','figure', 'chapter','abbcbccabcabcabcabcbcbabacba','abcdefghijklmnopqrstuvwxyz']

Se añaden las palabras que consideramos al conjunto principal

In [9]:
stopWords.extend(listoStopWords)

In [10]:
dictonary = nltk.corpus.words.words()

### Función principal procesamiento

In [11]:
def clean_files(texto,stopWords):
    
    #Quitar todos los acentos
    #texto = unidecode.unidecode(texto)
    
    #Quitar todos los caracteres especiales
    texto = re.sub('[^A-Za-z0-9]+',' ',texto)
    
    #Pasar todo a minisculas
    texto = texto.lower()
    
    #Tokenizar
    tokens = texto.split()
    
    tokens2 = [w for w in tokens if (len(w)>1)&(w.isalpha())&(w not in stopWords)]
    
    #Lematización
    word_net_lemmatizar = WordNetLemmatizer()

    tokens3 = [word_net_lemmatizar.lemmatize(w, pos = "v") for w in tokens2]
   
    #Stemmer
    ps = PorterStemmer() 
    tokens4 = [ps.stem(w) for w in tokens3]
     
    #Se retorna el texto nuevamente en un solo string luego de ser procesado
    to_return = ' '.join(tokens4)
    
    #Se retorna el vocabulario de cada documento
    set_words = set(tokens4)
    
    #Y la frecuencia de las palabras
    freq = nltk.FreqDist(tokens4)
    return to_return,set_words,freq

Inicialización de los conjuntos:

    - Vocabulary: el conjunto de todas las palabras que contienen los documentos
    - results_text: la lista con los documentos ya organizados para construir el bag of words
    - results_frecuency: información de cada documento de las palabras que contiene cuántas veces las contiene

In [12]:
vocabulary = set()
results_text = []
results_frecuency = []

## Limpieza de los datos

In [13]:
fileSummary = "CleanSummary.csv"

contenido= "Archivo" + ";" + "Tamaño(K)" + ";" + "Cant Palabras Inicial" + ";" + "Cant Palabras depuradas"+ ";" +"Porc Limpieza"+ "\n"
resultado = pd.DataFrame()
indexFiles = []
documents = []

for f in input_file_path.glob('*.txt'):
    #Peso del archivo
    tmpSize=round(fileSize(f)/1014)
    
    #Lectura del archivo
    input_file = open(f, "r", encoding = 'utf-8')
    input_aux = input_file.read()
    
    #Cuenta de palabras iniciales
    tmpWordsOri=wordsCount(input_aux)
    #out = datasetOut / str(f).split('/')[-1]
    
    #Cuenta de palabras despues de la limpieza
    tmpWordsEnd,text,setWord=cleanWordCount(input_aux)
    #tmpPerClean=round((tmpWordsEnd/tmpWordsOri)*100)
    
    
    #Detectando el idioma
    aux = detect_language(text)
    
    if(aux == 'english'):
        text_cleanned,set_words,freq = clean_files(text,stopWords)
        #out2 = datasetOut / str(f).split('/')[-1]
        documents.append(text_cleanned)
        vocabulary = vocabulary.union(set_words)
        results_text.append(text_cleanned)
        indexFiles.append(str(f).split('\\')[-1])
        
        #Escritura de los resultados del preprocesamiento
        auxRes= pd.DataFrame({'Archivo': str(f).split('/')[-1], 'Tamaño(K)': [tmpSize], 'Cant Palabras Inicial': [tmpWordsOri],'Cant Palabras depuradas': [tmpWordsEnd],"Vocabulario Inicial":len(setWord),"Vocabulario Final":len(set_words)})
        resultado = pd.concat([resultado, auxRes])
        #break
    else:
        print(aux + ':' + str(f))

resultado.to_csv(fileSummary, sep = ';', index = False)

german:E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almacenamiento\Tika\Tika\limpiados\1508.02340.txt


In [14]:
resultado.head()

,Archivo,Tamaño(K),Cant Palabras Inicial,Cant Palabras depuradas,Vocabulario Inicial,Vocabulario Final
0,E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almace...,14,2765,2765,515,326
0,E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almace...,22,3694,3694,1036,633
0,E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almace...,30,5482,5482,1254,798
0,E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almace...,94,18304,18304,1771,1057
0,E:\OneDrive - CELSIA S.A E.S.P\Maestría\Almace...,32,6227,6227,986,616


In [15]:
len(vocabulary)

49881

## Construcción del Bag of Words

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import vstack,save_npz,load_npz

Construido el vocabulario podemos construir el bag of words, que se hace con la ayuda de la funcion CountVectorizer

In [17]:
vectorizer = CountVectorizer(analyzer = "word",vocabulary =vocabulary , tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000) 
train_data_features = vectorizer.fit_transform(documents)

In [18]:
len(vocabulary)

49881

In [19]:
vectorizer.vocabulary_

{'aa': 0,
 'aaa': 1,
 'aab': 2,
 'aabab': 3,
 'aababa': 4,
 'aababaababb': 5,
 'aabak': 6,
 'aabb': 7,
 'aachen': 8,
 'aaecc': 9,
 'aaex': 10,
 'aagea': 11,
 'aai': 12,
 'aakv': 13,
 'aalborg': 14,
 'aall': 15,
 'aalto': 16,
 'aam': 17,
 'aama': 18,
 'aan': 19,
 'aand': 20,
 'aarabi': 21,
 'aaron': 22,
 'aaronson': 23,
 'aarti': 24,
 'aat': 25,
 'aatu': 26,
 'aatuut': 27,
 'aau': 28,
 'aaw': 29,
 'aawcondit': 30,
 'aax': 31,
 'aazhang': 32,
 'ab': 33,
 'aba': 34,
 'abab': 35,
 'ababa': 36,
 'ababac': 37,
 'ababb': 38,
 'abac': 39,
 'abacaxi': 40,
 'abad': 41,
 'abadi': 42,
 'abak': 43,
 'abalon': 44,
 'aban': 45,
 'abandon': 46,
 'abaqu': 47,
 'abasekmt': 48,
 'abb': 49,
 'abba': 50,
 'abbaa': 51,
 'abbaac': 52,
 'abbasi': 53,
 'abbeel': 54,
 'abboud': 55,
 'abbr': 56,
 'abbrevi': 57,
 'abbreviatetb': 58,
 'abc': 59,
 'abcab': 60,
 'abcd': 61,
 'abd': 62,
 'abdel': 63,
 'abdelgawad': 64,
 'abdelhak': 65,
 'abdoulay': 66,
 'abduct': 67,
 'abdul': 68,
 'abdullah': 69,
 'abdur': 70,
 'abe

In [20]:
txt_path = Path('E:/OneDrive - CELSIA S.A E.S.P/Maestría/Almacenamiento/Tika/Tika/limpiados')

file1 = open("E:/OneDrive - CELSIA S.A E.S.P/Maestría/Almacenamiento/proyectointegrador1/data/cranfield/cranfield.dat","w", encoding="utf-8")
indexMeta = {}
i = 0
for file in txt_path.glob('*.txt'):
    #Leer Informacion
    auxkey = str(file).split('\\')[-1]
    indexMeta[i] = auxkey
    i = i+1
    input_file = open(file,"r",encoding='utf-8')
    texto = input_file.read().replace('\n',' ')
    texto = re.sub('[^A-Za-z0-9]+',' ',texto).lower()
    file1.writelines(texto+'\n')
file1.close()

In [21]:
estructuraFinal = {'vectorizer' : vectorizer, 'idexFiles' :indexFiles, 'matriz':train_data_features, 'metapyIndex':indexMeta}

In [22]:
with open('vocabulary.txt', 'w') as f:
    for item in vectorizer.get_feature_names():
        f.write("%s\n" % item)

In [23]:
import pickle
filename = 'estructuraDatos.sav'
pickle.dump(estructuraFinal, open(filename, 'wb'))
#loaded_model = pickle.load(open('BoW1.sav', 'rb'))

In [26]:
len(documents[0])

8469

In [28]:
Total = 0
for doc in documents:
    Total = Total +len(doc)

In [30]:
Prom = Total/len(documents)
Prom

27544.226762002043